# **Framework Improvements - Adding The Learner Class**

In [3]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle, gzip, math, os, time, shutil
import fastcore.all as fc
from operator import attrgetter, itemgetter
from functools import partial
from collections.abc import Mapping
from contextlib import contextmanager
from pathlib import Path
from copy import copy

import torch
from torch import nn, tensor, optim
from torch.utils.data import default_collate, DataLoader
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from datasets import load_dataset, load_dataset_builder

from miniai.training import * # Modules have already been developed in previous NBs
from miniai.datasets import * # and, reside in separate Github repo.
from miniai.conv import *
from fastprogress import progress_bar, master_bar

In [2]:
from fastcore.test import test_close

torch.set_printoptions(precision=2, linewidth=14, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

import logging 
logging.disable(logging.WARNING)

## **The Learner**

The objective of this new class will be to:
1. Enable rapid testing of new approaches wrt the modeling process, especially on the training and inference fronts.
2. Constantly build on-top of the existing learner's functionality.
3. Allow improved inspection, optimized CUDA implementations etc.

So, let's get the HuggingFace `Fashion_MNIST` dataset again.

In [4]:
x, y = 'image', 'label'
name  = "fashion_mnist"
dsd = load_dataset(name)

In [5]:
dsd

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [6]:
@inplace
def transformi(b): b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [7]:
bs = 1024
tds = dsd.with_transform(transformi)

In [8]:
dls = DataLoaders.from_dd(tds, bs, num_workers=4)
dt = dls.train
xb, yb = next(iter(dt))
xb.shape, yb[:10]

(torch.Size([1024, 784]),
 tensor([5, 4,
         9, 4,
         3, 0,
         6, 5,
         7, 6]))

Prose

In [20]:
class Learner:
    def __init__(self, model, dls, loss_func, lr, opt_func=optim.SGD): 
        # fastcore store_attr() reduces the amount of boilerplate that usually goes
        # into an __init__()
        fc.store_attr()

    def one_batch(self):
        self.xb, self.yb = to_device(self.batch)
        self.preds = self.model(self.xb)
        self.loss = self.loss_func(self.preds, self.yb)
        if self.model.training:
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()
        with torch.no_grad(): self.calc_stats()

    def calc_stats(self):
        acc = (self.preds.argmax(dim=1)==self.yb).float().sum()
        self.accs.append(acc)
        n = len(self.xb)
        self.losses.append(self.loss*n)
        self.ns.append(n)

    def one_epoch(self, train):
        self.model.training = train
        dl = self.dls.train if train else self.dls.valid
        for self.num,self.batch in enumerate(dl): self.one_batch()
        n = sum(self.ns)
        print(self.epoch, self.model.training, sum(self.losses).item()/n, sum(self.accs).item()/n)
    
    def fit(self, n_epochs):
        self.accs, self.losses,self.ns = [],[],[]
        self.model.to(def_device)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        self.n_epochs = n_epochs
        for self.epoch in range(n_epochs):
            self.one_epoch(True)
            with torch.no_grad(): self.one_epoch(False)

We can test the new learner on the simple MLP. Also, note that we can now add `num_workers` to the DataLoader to improve the performance of collate functions.

In [17]:
m, nh = 28*28, 50
model = nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, 10))

In [18]:
learn = Learner(model, dls, F.cross_entropy, lr=0.2)
learn.fit(1)

0 True 1.1632997395833333 0.6137666666666667
0 False 1.1239921875 0.6265
